In [1]:
#Download necessary dependencies, only need to do once
#import sys
#!{sys.executable} -m pip install spacy
#!{sys.executable} -m spacy download en_core_web_lg
#!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
#!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz

import csv
import spacy
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import tokenize
STOPWORDS = set(stopwords.words('english'))
import numpy as np
import pandas as pd

In [2]:
# load spaCy models
nlp = spacy.load('en_core_web_lg')
nlpSci = spacy.load("en_ner_bc5cdr_md")
nlpSci2 = spacy.load('en_ner_bionlp13cg_md')

In [3]:
df = pd.read_csv('positive_abstract_set.csv')
pmids = df['pmid'].tolist()
abstract_test = df['abstract'].tolist()

In [4]:
for i in range(0,2):
    print(pmids[i])
    print(abstract_test[i])

31218158
In order to ascertain the current status of and challenges posed by the incidence and prevalence of rare diseases in China, this study teases out data on the incidence and prevalence of 121 rare diseases listed in <i>China's First List of Rare Disease</i> to provide rationales and references for the development and promotion of rare-disease-related policies. The National Health Commission of the People's Republic of China issued the <i>Rare Disease Diagnosis and Treatment Guide (2019)</i> (denoted here as <i>China's Rare Disease Diagnosis and Treatment Guide</i>), which cited data on the incidence/prevalence of 21 rare diseases (21 of 121 rare diseases, 17.36%). Data on 68 diseases (56.20%) were found in monographs, literature databases, and official websites. Data on the incidence/prevalence of 70 diseases were compiled, though no data were available for the 51 remaining diseases. There are published data on the incidence/prevalence of only 14 diseases at the national level. 

In [5]:
def remove_stopwords(string):
    for word in STOPWORDS:
        token = ' ' + word + ' '
        string = string.replace(token, ' ')
        string = string.strip()
    return string

In [6]:
abstracts =[]
for paragraph in df['abstract']:
    paragraph = remove_stopwords(paragraph)
    abstracts.append(paragraph)

In [7]:
for i in range(0,2):
    print(pmids[i])
    print(abstracts[i])

31218158
In order ascertain current status challenges posed incidence prevalence rare diseases China, study teases data incidence prevalence 121 rare diseases listed <i>China's First List Rare Disease</i> provide rationales references development promotion rare-disease-related policies. The National Health Commission People's Republic China issued <i>Rare Disease Diagnosis Treatment Guide (2019)</i> (denoted <i>China's Rare Disease Diagnosis Treatment Guide</i>), cited data incidence/prevalence 21 rare diseases (21 121 rare diseases, 17.36%). Data 68 diseases (56.20%) found monographs, literature databases, official websites. Data incidence/prevalence 70 diseases compiled, though data available 51 remaining diseases. There published data incidence/prevalence 14 diseases national level. Sources data incidence prevalence rare diseases mainly include cases counts hospitals (40.56%), sources data (24.48%), screening (20.98%), cross-sectional studies (8.39%), estimates models (7.69%). Data 

In [8]:
# Standardize the text of the abstract by replacing all named entities with their entity label
# Eg: 3 patients were seen in a clinic in England --> CARDINAL patients were seen in a clinic in GPE
def standardizeAbstract(abstract):
    doc = nlp(abstract)
    newAbstract = abstract
    # interate through the entities in the abstract
    for e in reversed(doc.ents):
        # replace entities with their label
        #GPE = GeoPolitical Entity
        if e.label_ in {'PERCENT','CARDINAL','GPE','LOC','DATE','TIME','QUANTITY','ORDINAL'}:
            start = e.start_char
            end = start + len(e.text)
            newAbstract = newAbstract[:start] + e.label_ + newAbstract[end:]
    return newAbstract

In [9]:
# Same process but include scientific entities from scispaCy models
def standardizeSciTerms(abstract):
    doc = nlpSci(abstract)
    newAbstract = abstract
    for e in reversed(doc.ents):
        start = e.start_char
        end = start + len(e.text)
        newAbstract = newAbstract[:start] + e.label_ + newAbstract[end:]
        
    doc = nlpSci2(newAbstract)
    for e in reversed(doc.ents):
        start = e.start_char
        end = start + len(e.text)
        newAbstract = newAbstract[:start] + e.label_ + newAbstract[end:]
    return newAbstract

In [10]:
#Turn abstract list into abstract list of sentences
#Tokenize abstract into list of sentences -shows as list within list
for i in range(len(abstracts)):
    abstracts[i] = tokenize.sent_tokenize(abstracts[i])

In [12]:
print(abstracts[5])
#print(abstracts[2])
print('3rd sentence')
print(abstracts[5][2])

['<h4>Purpose review</h4>Charcot-Marie-Tooth disease (CMT) one commonest inherited neuromuscular diseases population prevalence 1 2500.', 'This review cover recent advances genetics pathomechanisms CMT leading development rational therapies.<h4>Recent findings</h4>Pathomechanistic therapeutic target advances CMT include identification ErbB receptor signalling pathway therapeutic target CMT1A pharmacological modification unfolded protein response CMT1B.', 'In CMT2D, due mutations glycyl-tRNA synthetase, vascular endothelial growth factor-mediated stimulation Nrp1 receptor identified therapeutic target.', 'Preclinical advances accompanied publication large natural history cohorts identification sensitive biomarker disease (muscle MRI) able detect disease progression CMT1A 1 year.<h4>Summary</h4>Advances next-generation sequencing technology, cell biology animal models CMT paving way rational treatments.', 'The combination robust natural history data identification sensitive biomarkers me

In [13]:
def isGeo(sentence):
    doc = nlp(sentence)
    # interate through the entities in the abstract
    for ent in doc.ents:
        if ent.label_ in {'GPE','LOC'}:
            return True
    return False

In [16]:
def whereGeo(sentence):
    locations = []
    doc = nlp(sentence)
    # interate through the entities in the sentence
    for ent in doc.ents:
        if ent.label_ in {'GPE','LOC'}:
            locations.append(ent.text)
    return locations

In [ ]:
print(abstracts[5][8])
location = whereGeo(abstracts[5][8])
print(location)
#abstracts[5][8] = whereGeo(abstracts[5][8])
#print(abstracts[5][8][0])


In [14]:
geopmids = []
geosentences = []
#Returns all of the sentences with location information
i=1
for abstract in abstracts:
    for sentence in abstract:
        if isGeo(sentence):
            geopmids.append(pmids[i])
            geosentences.append(sentence)
    i+=1

In [ ]:
#this one replaces all sentences with lists of the location data
i=0
j=0
for abstract in abstracts:
    for sentence in abstract:
        abstracts[i][j] = whereGeo(str(sentence))
        j+1
    i+=1

In [17]:
locInfo = {}

i=0
for abstract in abstracts:
    for sentence in abstract:
        if isGeo(sentence):
            if(i>6):
                break
            print(i)
            print(pmids[i])
            print(sentence)
            print(whereGeo(sentence))
            print('............................')
            locInfo[pmids[i]]=whereGeo(sentence)
            #pmidlocs.append(pmids[i])
            #locInfo.append(whereGeo(str(sentence)))
    i+=1

0
31218158
In order ascertain current status challenges posed incidence prevalence rare diseases China, study teases data incidence prevalence 121 rare diseases listed <i>China's First List Rare Disease</i> provide rationales references development promotion rare-disease-related policies.
['China']
............................
0
31218158
Data incidence/prevalence rare diseases China limited typically lack accuracy, uniformity, timeliness.
['China']
............................
0
31218158
China recently initiated epidemiological studies rare diseases national regional level.
['China']
............................
1
33985607
Epidemiological data human brucellosis Spain scarce.
['Spain']
............................
1
33985607
The objective study assess epidemiological characteristics inpatient brucellosis Spain 1997 2015.
['Spain']
............................
2
33754462
<h4>Background</h4>Few data available prevalence neural tube defects (NTDs) within different ethnic communities United

In [18]:
print(locInfo)

{31218158: ['China'], 33985607: ['Spain'], 33754462: ['England Wales'], 33748277: ['Pseudoachondroplasia'], 31788425: ['Nepal']}


In [19]:
print(pmidlocs[0])
#print(abstracts[0])
#print(abstracts[0][0])
print(locInfo[0])

NameError: name 'pmidlocs' is not defined

In [20]:
print(abstracts[0])
print('----------')
print(abstracts[0][0])
print('----------')
print(abstracts[0][0][0])

["In order ascertain current status challenges posed incidence prevalence rare diseases China, study teases data incidence prevalence 121 rare diseases listed <i>China's First List Rare Disease</i> provide rationales references development promotion rare-disease-related policies.", "The National Health Commission People's Republic China issued <i>Rare Disease Diagnosis Treatment Guide (2019)</i> (denoted <i>China's Rare Disease Diagnosis Treatment Guide</i>), cited data incidence/prevalence 21 rare diseases (21 121 rare diseases, 17.36%).", 'Data 68 diseases (56.20%) found monographs, literature databases, official websites.', 'Data incidence/prevalence 70 diseases compiled, though data available 51 remaining diseases.', 'There published data incidence/prevalence 14 diseases national level.', 'Sources data incidence prevalence rare diseases mainly include cases counts hospitals (40.56%), sources data (24.48%), screening (20.98%), cross-sectional studies (8.39%), estimates models (7.69%

In [ ]:
for i in range(len(geosentences)):
    print(geopmids[1-i])
    print(geosentences[1-i])

In [ ]:
print(geopmids)
print(geosentences)